In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np ;
from keras.utils import np_utils
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Reshape,Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.models import Model
import itertools
import cv2
import pandas as pd ;
import matplotlib.pyplot as plt ;
import tensorflow as tf
from tensorflow.keras.applications import VGG16 , ResNet50 , Xception , InceptionV3
from tensorflow.keras.layers import Flatten
import pandas as pd
import os

# Combining All Features

In [ ]:
#Reading from csv files if size is less (less ram consumed)

dataset1 = pd.read_csv("/content/drive/MyDrive/STAAR_PROJECT/alzhiemer/Minor_Project/Sudeb160/traincombined1.csv")
dataset2 = pd.read_csv("/content/drive/MyDrive/STAAR_PROJECT/alzhiemer/Minor_Project/Sudeb160/trainvgg16.csv")

In [ ]:
#If csv files are huge, divide data in chunks and concat them individually

#Defining paths and chunksize. Chunksize is how many rows will be there in one chunk
path1="/content/drive/MyDrive/STAAR_PROJECT/alzhiemer/Minor_Project/Sudeb160/traincombined3.csv"
path2="/content/drive/MyDrive/STAAR_PROJECT/alzhiemer/Minor_Project/Sudeb160/trainxecption.csv"
chunksize = 100

#Getting dataset1 via chunk method
tfr1 = pd.read_csv(path1, chunksize=chunksize, iterator=True,header= None)
dataset1 =[]
dataset1 = pd.DataFrame(dataset1)
i=1
with tfr1 as reader:
    for chunk in reader:
        print("index_value")
        print(i)
        dataset1 = pd.concat([dataset1,chunk], axis=0, ignore_index=True)
        i=i+1

#Getting dataset2 via chunk method
tfr2 = pd.read_csv(path2, chunksize=chunksize, iterator=True,header= None)
dataset2=[]
dataset2= pd.DataFrame(dataset2)
i=1
with tfr2 as reader:
    for chunk in reader:
        print("index_value")
        print(i)
        dataset2 = pd.concat([dataset2,chunk], axis=0, ignore_index=True)
        i=i+1


In [ ]:
#visualize dataset1
dataset1

,0,1,2,3,4,5,6,7,8,9,...,276984,276985,276986,276987,276988,276989,276990,276991,276992,276993
1,0,021aug1.bmp,0.0,0.000000,0.0,17.682007,16.030294,0.000000,32.785278,34.955849,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,021aug2.bmp,0.0,9.120188,0.0,35.176334,0.000000,2.535757,40.231358,39.784996,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,021aug3.bmp,0.0,0.000000,0.0,0.000000,20.211481,0.000000,0.000000,11.552873,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,022aug2.bmp,0.0,3.189520,0.0,31.731390,0.000000,17.184370,40.031578,41.260105,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0,022aug3.bmp,0.0,0.000000,0.0,6.088952,22.149429,0.000000,0.000000,17.969427,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,1,019aug1.bmp,0.0,0.000000,0.0,0.000000,28.385202,0.000000,0.463453,30.069345,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
381,1,019aug2.bmp,0.0,0.000000,0.0,5.902587,6.514037,0.000000,9.817458,31.492180,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
382,1,020aug1.bmp,0.0,0.000000,0.0,0.000000,32.366978,0.000000,0.000000,28.413380,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
383,1,020aug2.bmp,0.0,0.000000,0.0,5.498646,19.613117,0.000000,10.515870,25.603579,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#visualize dataset2
dataset2

,0,1,2,3,4,5,6,7,8,9,...,100344,100345,100346,100347,100348,100349,100350,100351,100352,100353
0,0,1,2.0,3.000000,4.0,5.0,6.0,7.0,8.000000,9.0,...,100344.0,100345.0,100346.0,100347.0,100348.0,100349.0,100350.0,100351.0,100352.0,100353.0
1,0,021aug1.bmp,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,021aug2.bmp,0.0,2.849242,0.0,0.0,0.0,0.0,1.637874,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,021aug3.bmp,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,022aug2.bmp,0.0,0.000000,0.0,0.0,0.0,0.0,1.791373,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,1,019aug1.bmp,0.0,0.000000,0.0,0.0,0.0,0.0,2.384850,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
381,1,019aug2.bmp,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
382,1,020aug1.bmp,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
383,1,020aug2.bmp,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#Optional Code. Run this if any of the two datasets have one extra row in the beginning due to not using header=None while saving the dataset
dataset1.drop(index=dataset1.index[0],
        axis=0,
        inplace=True)

In [ ]:
#merge dataset2 on the right of dataset 1 while taking dataset2.iloc[:,1] i.e 2nd column as key.
#2nd colum contains name of files which is common in both datasets and combines rows with same file name
df = dataset1.merge(dataset2, on = dataset2.iloc[:,1], how = 'left')

#visualize the dataframe
df

,key_0,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,...,100344_y,100345_y,100346_y,100347_y,100348_y,100349_y,100350_y,100351_y,100352_y,100353_y
0,1,0,1,2.0,3.000000,4.0,5.000000,6.000000,7.000000,8.000000,...,100344.0,100345.0,100346.0,100347.0,100348.0,100349.0,100350.0,100351.0,100352.0,100353.0
1,021aug1.bmp,0,021aug1.bmp,0.0,0.000000,0.0,17.682007,16.030294,0.000000,32.785278,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,021aug2.bmp,0,021aug2.bmp,0.0,9.120188,0.0,35.176334,0.000000,2.535757,40.231358,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,021aug3.bmp,0,021aug3.bmp,0.0,0.000000,0.0,0.000000,20.211481,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,022aug2.bmp,0,022aug2.bmp,0.0,3.189520,0.0,31.731390,0.000000,17.184370,40.031578,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,019aug1.bmp,1,019aug1.bmp,0.0,0.000000,0.0,0.000000,28.385202,0.000000,0.463453,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
381,019aug2.bmp,1,019aug2.bmp,0.0,0.000000,0.0,5.902587,6.514037,0.000000,9.817458,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
382,020aug1.bmp,1,020aug1.bmp,0.0,0.000000,0.0,0.000000,32.366978,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
383,020aug2.bmp,1,020aug2.bmp,0.0,0.000000,0.0,5.498646,19.613117,0.000000,10.515870,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# check where are extra values in this new combined dataset df.
# for example if dataset1 had 10,000 columns then dataset2 is added.
# then dataset2 also has 2 extra columns i.e label and file names.
# these need to be deleted since dataset1 already has them covered.

#To check this, start with columns where dataset1 ends.
# Example: if dataset1 had 10000 columns then dataset2 is added,
# then in combined dataset df, values from 9999th to 10005th may have extra values

#check for them like this

df.iloc[:,[0,1,176643,176644]]

,key_0,0_x,0_y,1_y
0,1,0,0,1
1,021aug1.bmp,0,0,021aug1.bmp
2,021aug2.bmp,0,0,021aug2.bmp
3,021aug3.bmp,0,0,021aug3.bmp
4,022aug2.bmp,0,0,022aug2.bmp
...,...,...,...,...
380,019aug1.bmp,1,1,019aug1.bmp
381,019aug2.bmp,1,1,019aug2.bmp
382,020aug1.bmp,1,1,020aug1.bmp
383,020aug2.bmp,1,1,020aug2.bmp


In [ ]:
#after finding those two extra columns, drop them.
#also drop first column(this is extra column created by merge funtion)
df =df.drop(df.columns[[0,176643,176644]],
                       axis = 1)
#column names are not 0,1,2,3,4,5... they are 0x,1x,2x etc etc.
#therefore these two lines change it
df = df[:].values
df=pd.DataFrame(df)

In [ ]:
#now updated dataset looks like this with first column as label/output, 2nd column as file name
# and rest of columns are combined features
df

,0,1,2,3,4,5,6,7,8,9,...,276984,276985,276986,276987,276988,276989,276990,276991,276992,276993
1,0,021aug1.bmp,0.0,0.000000,0.0,17.682007,16.030294,0.000000,32.785278,34.955849,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,021aug2.bmp,0.0,9.120188,0.0,35.176334,0.000000,2.535757,40.231358,39.784996,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,021aug3.bmp,0.0,0.000000,0.0,0.000000,20.211481,0.000000,0.000000,11.552873,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,022aug2.bmp,0.0,3.189520,0.0,31.731390,0.000000,17.184370,40.031578,41.260105,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0,022aug3.bmp,0.0,0.000000,0.0,6.088952,22.149429,0.000000,0.000000,17.969427,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,1,019aug1.bmp,0.0,0.000000,0.0,0.000000,28.385202,0.000000,0.463453,30.069345,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
381,1,019aug2.bmp,0.0,0.000000,0.0,5.902587,6.514037,0.000000,9.817458,31.492180,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
382,1,020aug1.bmp,0.0,0.000000,0.0,0.000000,32.366978,0.000000,0.000000,28.413380,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
383,1,020aug2.bmp,0.0,0.000000,0.0,5.498646,19.613117,0.000000,10.515870,25.603579,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#optional code, see if an extra header row is added at top. if yes then remove it else ignore this code
df.drop(index=df.index[0],
        axis=0,
        inplace=True)

In [ ]:
#save your project
df.to_csv("/content/drive/MyDrive/STAAR_PROJECT/alzhiemer/Minor_Project/Sudeb160/traincombined3.csv", index= False, header= None)